In [42]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#from data import VOC_ROOT, VOCAnnotationTransform, VOCDetection, BaseTransform
from data import VISAPP_ROOT, AnnotationTransform, VISAPP2020_Detection, BaseTransform
from data import VISAPP2020_CLASSES as labelmap
import torch.utils.data as data

from ssd import build_ssd

import sys
import os
import time
import argparse
import numpy as np
import pickle
import cv2
import json

In [43]:
labelmap = (  # always index 0
    'fork', 'knife', 'ladle',
    'mug', 'peeler', 'spatula', 'spoon', 'hammer',
    'scissors', 'shovel')

In [44]:
if sys.version_info[0] == 2:
    import xml.etree.cElementTree as ET
else:
    import xml.etree.ElementTree as ET

set_type = "test"

In [45]:
def do_python_eval(output_dir='output', use_07=True):
    cachedir = os.path.join(devkit_path, 'annotations_cache')
    aps = []
    # The PASCAL VOC metric changed in 2010
    use_07_metric = use_07
    print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, cls in enumerate(labelmap):
        filename = get_voc_results_file_template(set_type, cls)
        rec, prec, ap = voc_eval(
           filename, annopath, imgsetpath.format(set_type), cls, cachedir,
           ovthresh=0.5, use_07_metric=use_07_metric)
        aps += [ap]
        print('AP for {} = {:.4f}'.format(cls, ap))
        with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
            pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
    print('Mean AP = {:.4f}'.format(np.mean(aps)))
    print('~~~~~~~~')
    print('Results:')
    for ap in aps:
        print('{:.3f}'.format(ap))
    print('{:.3f}'.format(np.mean(aps)))
    print('~~~~~~~~')
    print('')
    print('--------------------------------------------------------------')
    print('Results computed with the **unofficial** Python eval code.')
    print('Results should be very close to the official MATLAB eval code.')
    print('--------------------------------------------------------------')

In [46]:
def get_voc_results_file_template(image_set, cls):
    # VOCdevkit/VOC2007/results/det_test_aeroplane.txt
    filename = 'det_' + image_set + '_%s.txt' % (cls)
    filedir = os.path.join(devkit_path, 'results')
    if not os.path.exists(filedir):
        os.makedirs(filedir)
    path = os.path.join(filedir, filename)
    return path

In [47]:
devkit_path = "/home/akizuki/VISAPP2020/Dataset/"
output_dir = "output"
use_07 = True

cachedir = os.path.join(devkit_path, 'annotations_cache')
print(cachedir)
aps = []
# The PASCAL VOC metric changed in 2010
use_07_metric = use_07
print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
for i, cls in enumerate(labelmap):
    filename = get_voc_results_file_template(set_type, cls)
    print(filename)

/home/akizuki/VISAPP2020/Dataset/annotations_cache
VOC07 metric? Yes
/home/akizuki/VISAPP2020/Dataset/results/det_test_fork.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_knife.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_ladle.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_mug.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_peeler.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_spatula.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_spoon.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_hammer.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_scissors.txt
/home/akizuki/VISAPP2020/Dataset/results/det_test_shovel.txt


## voc_eval()の理解

In [75]:
def get_voc_results_file_template(image_set, cls):
    # VOCdevkit/VOC2007/results/det_test_aeroplane.txt
    filename = 'det_' + image_set + '_%s.txt' % (cls)
    filedir = os.path.join(devkit_path, 'results')
    if not os.path.exists(filedir):
        os.makedirs(filedir)
    path = os.path.join(filedir, filename)
    return path

In [76]:
filename = get_voc_results_file_template(set_type, labelmap[1])
print(filename, "exist? ", os.path.isfile(filename))

/home/akizuki/VISAPP2020/Dataset/results/det_test_knife.txt exist?  True


In [77]:
import glob
#voc_evalの引数
cachedir = os.path.join(devkit_path, 'annotations_cache')
annopath = os.path.join(devkit_path, '%s', 'DetectionAnnotations', 'bbox%s.json')
imgpath = os.path.join(devkit_path,  '%s', 'RGBImages', 'im_color%s.png')
imgsetpath = os.path.join(devkit_path, 'ImageSets', '{:s}.txt')

#### 各BBox名を取得するためのコード

In [78]:
imagesets_list = list() #trainかtestのリストを作る．[train,test].txtにはSceneXXXが書かれている．
for t in open(os.path.join(devkit_path, 'test.txt') ):
    imagesets_list.append(t.rstrip('\n')) #改行コードを削除する

ids = list()
for scene in imagesets_list:
    rootpath = os.path.join(devkit_path, scene )
    filelist = glob.glob( os.path.join(rootpath,'DetectionAnnotations','*') )
    
    for line in filelist:
        #"パス"，"5桁のインデクス" のペアを記録する．
        #bbox基準に探すことで，アノテーションに失敗した画像を読み込まないようにする．
        ids.append((line[:-36], line[-10:-5])) 

In [79]:
def parse_rec_visapp(filename):
    """ Parse a VISAPP json file """
    anno = open( filename )
    target = json.load(anno)
    
    objects = []
    for cls_id in range(len(labelmap)):
        if labelmap[cls_id] in target['Annotation']:
            obj_struct = {}
            key = labelmap[cls_id]
            obj_struct['name'] = key
            obj_struct['difficult'] = 0 #とりあえず0を入れておく
            obj_struct['bbox'] = [int(target['Annotation'][key][0]["xmin"])-1,
                                  int(target['Annotation'][key][0]["ymin"])-1,
                                  int(target['Annotation'][key][0]["xmax"])-1,
                                  int(target['Annotation'][key][0]["ymax"])-1]
            objects.append(obj_struct)

    return objects

In [80]:
obj = parse_rec_visapp("bbox00000.json")

In [81]:
obj

[{'name': 'ladle', 'difficult': 0, 'bbox': [227, 79, 370, 294]},
 {'name': 'mug', 'difficult': 0, 'bbox': [153, 79, 286, 226]},
 {'name': 'spatula', 'difficult': 0, 'bbox': [321, 142, 615, 246]},
 {'name': 'spoon', 'difficult': 0, 'bbox': [245, 258, 467, 388]},
 {'name': 'hammer', 'difficult': 0, 'bbox': [107, 102, 242, 366]}]

In [82]:
if not os.path.isdir(cachedir):
    os.mkdir(cachedir)
cachefile = os.path.join(cachedir, 'annots.pkl')

In [110]:
# read list of images
with open(imgsetpath.format("test"), 'r') as f:
    lines = f.readlines()

# Scene idと画像idのリストの取り出し
tmps = [x.strip() for x in lines]
scene_ids = [ x[0:8] for x in tmps ]
img_ids = [ x[9:] for x in tmps ]
imagenames = [ x[0:8]+"_"+x[9:] for x in tmps ]

In [111]:
if not os.path.isfile(cachefile):
    # load annots
    recs = {}
    for i, (scene_id, img_id) in enumerate(zip(scene_ids, img_ids)):
        #annopathを指定して，画像に対するBBoxを取得する
        #print(i, scene_id, img_id)
        imagename = scene_id+"_"+img_id
        recs[imagename] = parse_rec_visapp(annopath % (scene_id, img_id))
        if i % 100 == 0:
            print('Reading annotation for {:d}/{:d}'.format(
               i + 1, len(scene_ids)))
    # save
    print('Saving cached annotations to {:s}'.format(cachefile))
    with open(cachefile, 'wb') as f:
        pickle.dump(recs, f)
else:
    # load
    print("load cache", cachefile)
    with open(cachefile, 'rb') as f:
        recs = pickle.load(f)

load cache /home/akizuki/VISAPP2020/Dataset/annotations_cache/annots.pkl


In [112]:
classname = labelmap[0]
# extract gt objects for this class
class_recs = {}
npos = 0
for imagename in imagenames:
    
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
                             'difficult': difficult,
                             'det': det}

In [91]:
detfile = filename.format(classname)

In [114]:
detfile

'/home/akizuki/VISAPP2020/Dataset/results/det_test_knife.txt'

In [115]:
with open(detfile, 'r') as f:
    lines = f.readlines()
if any(lines) == 1:
    #det_(test or train)_(クラス名).txtの読み込み
    #シーンID，画像ID，confidence, BBoxの順に読み取る．
    splitlines = [x.strip().split(' ') for x in lines]
    dataset_ids = [x[0] for x in splitlines]
    image_ids = [x[1] for x in splitlines]
    confidence = np.array([float(x[2]) for x in splitlines])
    BB = np.array([[float(z) for z in x[3:]] for x in splitlines])

    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]    
    dataset_ids = [dataset_ids[x] for x in sorted_ind]
    
    sorted_
    
    
    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    print(nd)
    for d in range(nd):
        try:
            R = class_recs[str(dataset_ids[d]+"_"+image_ids[d])] #SceneXXX_YYYYY(<-image id)にする
            #R = class_recs["Scene070_00102"] #SceneXXX_YYYYY(<-image id)にする
            bb = BB[d, :].astype(float)
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float)
            if d ==0:
                print(str(dataset_ids[d]+"_"+image_ids[d]))
                print(class_recs[str(dataset_ids[d]+"_"+image_ids[d])])
                print(R)
                print(bb)
                print(BBGT)
        except KeyError:
            fp[d] = 1

867
Scene070_00102
{'bbox': array([[288, 198, 428, 304]]), 'difficult': array([False], dtype=bool), 'det': [False]}
{'bbox': array([[288, 198, 428, 304]]), 'difficult': array([False], dtype=bool), 'det': [False]}
[ 147.   172.7  399.8  314.3]
[[ 288.  198.  428.  304.]]


In [97]:
fp = np.cumsum(fp)
fp

array([   1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,    9.,
         10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,   18.,
         19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,   27.,
         28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,   36.,
         37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,   45.,
         46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,   54.,
         55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,   63.,
         64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,   72.,
         73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,   81.,
         82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,   90.,
         91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,   99.,
        100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,  108.,
        109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,  117.,
        118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,  126.,
        127.,  128.,

In [252]:
image_ids

['00420',
 '00023',
 '00358',
 '00076',
 '00432',
 '00024',
 '00460',
 '00465',
 '00003',
 '00415',
 '00424',
 '00357',
 '00174',
 '00549',
 '00438',
 '00418',
 '00021',
 '00020',
 '00422',
 '00466',
 '00462',
 '00431',
 '00427',
 '00416',
 '00073',
 '00077',
 '00550',
 '00433',
 '00484',
 '00543',
 '00425',
 '00419',
 '00191',
 '00426',
 '00429',
 '00430',
 '00428',
 '00436',
 '00453',
 '00548',
 '00463',
 '00434',
 '00186',
 '00437',
 '00464',
 '00448',
 '00457',
 '00179',
 '00441',
 '00542',
 '00188',
 '00544',
 '00307',
 '00190',
 '00070',
 '00185',
 '00001',
 '00414',
 '00311',
 '00000',
 '00069',
 '00002',
 '00197',
 '00547',
 '00074',
 '00203',
 '00435',
 '00331',
 '00083',
 '00449',
 '00359',
 '00417',
 '00333',
 '00451',
 '00078',
 '00189',
 '00308',
 '00412',
 '00362',
 '00485',
 '00440',
 '00066',
 '00355',
 '00178',
 '00022',
 '00483',
 '00068',
 '00421',
 '00345',
 '00446',
 '00354',
 '00017',
 '00184',
 '00347',
 '00315',
 '00071',
 '00084',
 '00458',
 '00364',
 '00334',


## ImageSetsの作成のためのスクリプト

In [232]:
import glob
import json
#クラス名の定義
VISAPP2020_CLASSES = (  # always index 0
    'fork', 'knife', 'ladle',
    'mug', 'peeler', 'spatula', 'spoon', 'hammer',
    'scissors', 'shovel')

In [233]:
imagesets_list = list() #trainかtestのリストを作る．[train,test].txtにはSceneXXXが書かれている．
for t in open(os.path.join(devkit_path, 'test.txt') ):
    imagesets_list.append(t.rstrip('\n')) #改行コードを削除する

ids = list()
for scene in imagesets_list:
    rootpath = os.path.join(devkit_path, scene )
    filelist = glob.glob( os.path.join(rootpath,'DetectionAnnotations','*') )
    
    for line in filelist:
        #"パス"，"5桁のインデクス" のペアを記録する．
        #bbox基準に探すことで，アノテーションに失敗した画像を読み込まないようにする．
        ids.append((line[-44:-36], line[-10:-5])) 

In [234]:
for cls_id in range(10):
    imageset = list()
    for id in ids:
        anno = open(annopath % id)
        target = json.load(anno)
        flag = -1 #まずは-1にする
        if VISAPP2020_CLASSES[cls_id] in target['Annotation']:
            flag = 1 #アノテーションの中にクラスが見つかったら1
        imageset.append( (id[0],id[1],flag) )
    
    fname = labelmap[cls_id]+"_test.txt"
    with open(fname, mode='w') as f:
        for i in imageset:
            data = str(i[0])+" "+str(i[1])+" "+str(i[2])+"\n"
            f.write(data)

fname = "test.txt"   
with open(fname, mode='w') as f:
    for i in imageset:
        data = str(i[0])+" "+str(i[1])+"\n"
        f.write(data)

==================================================================================